# Multiplayer colors

A long time ago I built a multiplayer game skeleton. Because I had a tough time picking a few colors for the players I instead _generated_ them using `HSB` by hardcoding `Saturation` and `Brightness` while the `Hue` bit was selected at random. Thankfully thats where I stopped and implemented some of the _game_. What if I were to continue today?

First let's define a couple of variables:

In [3]:
var n = 4;
var speed = 1;

In [13]:
function clamp(value, min, max) {
  return Math.max(Math.min(value, max), min);
}

function rgbToHex({r, g, b}) {
  return "#" + (b | (g << 8) | (r << 16) | (1 << 24)).toString(16).slice(1);
}

const linearToGamma = (c) =>
  c >= 0.0031308 ? 1.055 * Math.pow(c, 1 / 2.4) - 0.055 : 12.92 * c;


function oklabToSRGB({L, a, b}) {
  var l = L + a * +0.3963377774 + b * +0.2158037573;
  var m = L + a * -0.1055613458 + b * -0.0638541728;
  var s = L + a * -0.0894841775 + b * -1.2914855480;
  // The ** operator here cubes; same as l_*l_*l_ in the C++ example:
  l = l ** 3; m = m ** 3; s = s ** 3;
  var r = l * +4.0767416621 + m * -3.3077115913 + s * +0.2309699292;
  var g = l * -1.2684380046 + m * +2.6097574011 + s * -0.3413193965;
  var b = l * -0.0041960863 + m * -0.7034186147 + s * +1.7076147010;
  // Convert linear RGB values returned from oklab math to sRGB for our use before returning them:
  r = 255 * linearToGamma(r); g = 255 * linearToGamma(g); b = 255 * linearToGamma(b);
  // OPTION: clamp r g and b values to the range 0-255; but if you use the values immediately to draw, JavaScript clamps them on use:
  r = clamp(r, 0, 255); g = clamp(g, 0, 255); b = clamp(b, 0, 255);
  // OPTION: round the values. May not be necessary if you use them immediately for rendering in JavaScript, as JavaScript (also) discards decimals on render:
    r = Math.round(r); g = Math.round(g); b = Math.round(b);
  return {r, g, b};
}

## The `setup` function

The usual p5 setup function, which creates the canvas.

In [1]:
function setup () {
  createCanvas(innerWidth, innerHeight);
  rectMode(CENTER);
}

## The `draw` function

From the [p5.js documentation](https://p5js.org/reference/#/p5/draw):

> The `draw()` function continuously executes the lines of code contained inside its block until the program is stopped or `noLoop()` is called.

In [26]:
function draw() {
  let c = oklabToSRGB({L: 0.65, a:-0.06, b: -0.18});
  c = oklabToSRGB({L: 0.65, a:0.20, b:0.12});
  background(rgbToHex(c));
  translate(innerWidth / 2, innerHeight / 2);
  for (let i = 0; i < n; i++) {
    push();
    rotate(frameCount * speed / 1000 * (i + 1));
    let f = oklabToSRGB({L: 0.2+i*(0.10), a:0.20, b:-0.18});
    fill(f.r, f.g, f.b);
    const s = 200 - i * 10;
    rect(0, 0, s, s);
    pop();
  }
}

In [19]:
%show